In [ ]:
!pip install datasets transformers torch seqeval torchcrf evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.2 MB/s eta 0:00:00
  

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tner/btc")

print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

btc.py:   0%|          | 0.00/3.91k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/494k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/77.7k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/180k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6338 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 6338
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 2000
    })
})


In [ ]:
print(dataset["train"][0])
print(dataset["train"].features[f"tags"])

{'tokens': ['I', 'hate', 'the', 'words', 'chunder', ',', 'vomit', 'and', 'puke', '.', 'BUUH', '.'], 'tags': [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]}
Sequence(feature=ClassLabel(names=['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O'], id=None), length=-1, id=None)


In [ ]:
label_list = dataset["train"].features[f"tags"].feature.names
label_list

['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O']

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,tokens,tags
0,"[Blake, 's, Jerusalem, ', reserved, for, homosexuals, ', http://tgr.ph/k5UU7g]","[B-PER, O, B-LOC, O, O, O, O, O, O]"
1,"[Retired, University, of, Sydney, media, lecturer, tells, all, about, bias, ., http://t.co/OwHGiiHY3E, http://t.co/AWTQa0TH47]","[O, B-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O]"
2,"[Shannon, Matthews, TV, drama, plans, causes, anger, in, Dewsbury, http://t.co/8K37hJwOj3]","[B-PER, I-PER, O, O, O, O, O, O, B-LOC, O]"
3,"[In, Irish, News, tmro, we, continue, our, #Giro, theme, with, a, chance, to, WIN, a, true, gusto, d, ', Italia, experience, in, @, CoppiBelfast, &, @, ilpiratabelfast]","[O, B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, B-LOC, O, B-LOC, B-LOC]"
4,"[BREAKING, :, Canada, 's, Milos, Raonic, falls, to, Roger, Federer, in, straight, sets, in, the, Wimbledon, semifinal, .]","[O, O, B-LOC, O, B-PER, I-PER, O, O, B-PER, I-PER, O, O, O, O, O, B-LOC, O, O]"


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-cased"
batch_size = 16
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
example = dataset["train"][4]
print(example["tokens"])
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['Think', 'you', 'call', 'that', 'smash', 'and', 'grab', '.', '#', 'Gateshead', "'s", 'media', 'man', 'just', 'admitted', 'to', 'me', 'it', 'was', '\'"\'', 'daylight', 'robbery', '\'"\'', '.', 'Shaw', "'s", 'only', 'touch', 'was', 'his', 'goal', '.']
['[CLS]', 'Think', 'you', 'call', 'that', 'smash', 'and', 'grab', '.', '#', 'Gates', '##head', "'", 's', 'media', 'man', 'just', 'admitted', 'to', 'me', 'it', 'was', "'", '"', "'", 'daylight', 'robbery', "'", '"', "'", '.', 'Shaw', "'", 's', 'only', 'touch', 'was', 'his', 'goal', '.', '[SEP]']


In [ ]:
len(example[f"tags"]), len(tokenized_input["input_ids"])

(32, 41)

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

41 41


In [ ]:
label_all_tokens = False
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(dataset['train'][:3])

{'input_ids': [[101, 146, 4819, 1103, 1734, 22572, 6775, 1200, 117, 26979, 1105, 23609, 2391, 119, 139, 2591, 2591, 3048, 119, 102], [101, 878, 119, 119, 114, 114, 113, 100, 119, 113, 100, 114, 100, 119, 878, 119, 208, 100, 221, 878, 119, 100, 119, 119, 100, 100, 119, 119, 100, 8325, 1203, 2381, 102], [101, 2586, 27376, 180, 7076, 4163, 182, 13505, 11622, 1161, 5682, 1324, 188, 10805, 20003, 3984, 18959, 7235, 9705, 1403, 112, 107, 112, 157, 10073, 1204, 8842, 2428, 9159, 131, 152, 1777, 1377, 5837, 5567, 1324, 170, 21270, 8914, 4267, 12477, 7609, 5871, 2047, 5358, 2047, 12921, 1161, 4267, 185, 27547, 5871, 2047, 119, 108, 157, 10073, 1204, 8842, 2428, 9159, 112, 107, 112, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/6338 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    #push_to_hub=True,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

<ipython-input-35-479ce2e20577>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: carinoalpha1997 (carinoalpha1997-university-of-bristol) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.156708
2,0.191900,0.153741
3,0.087600,0.163460


TrainOutput(global_step=1191, training_loss=0.12920419275009962, metrics={'train_runtime': 12551.8995, 'train_samples_per_second': 1.515, 'train_steps_per_second': 0.095, 'total_flos': 547732361096580.0, 'train_loss': 0.12920419275009962, 'epoch': 3.0})

In [ ]:
import numpy as np
from seqeval.metrics import classification_report

def compute_metrics(p):
    predictions, labels, _ = trainer.predict(p)
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return classification_report(true_labels, true_predictions)

print("Test set evaluation:")
print(compute_metrics(tokenized_datasets["test"]))

Test set evaluation:


              precision    recall  f1-score   support

         LOC       0.73      0.65      0.69       636
         ORG       0.60      0.57      0.58      1090
         PER       0.90      0.87      0.88      2650

   micro avg       0.80      0.76      0.78      4376
   macro avg       0.74      0.70      0.72      4376
weighted avg       0.80      0.76      0.78      4376

